In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

Implémentation de l'environnement TicTacToe

In [ ]:
import random

class TicTacToe:
    def __init__(self):
        self.reset()

    def reset(self):
        self.board = [' '] * 9
        self.current_player = 'X' 
        return self.board.copy()

    def step(self, action):
        if self.board[action] != ' ':
            return self.board.copy(), -10, True, {"invalid": True}

        self.board[action] = self.current_player
        if self.check_win(self.current_player):
            return self.board.copy(), 1, True, {}

        if ' ' not in self.board:
            return self.board.copy(), 0, True, {}

        # adversaire random
        opponent_move = random.choice([i for i, v in enumerate(self.board) if v == ' '])
        self.board[opponent_move] = 'O'
        if self.check_win('O'):
            return self.board.copy(), -1, True, {}

        return self.board.copy(), 0, False, {}

    def check_win(self, player):
        wins = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
        return any(all(self.board[i] == player for i in combo) for combo in wins)

    def render(self):
        for i in range(3):
            print(self.board[3*i:3*(i+1)])
